In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from gurobipy import *
import scipy.stats as ss

In [2]:
y = pd.read_excel("example.xlsx", sheet_name="Sheet3", index_col=0)
px = np.array([0.2]*5)
y = np.array(y)
py = y.sum(axis=0)
py = py/sum(py)

In [3]:
py

array([0.312, 0.128, 0.12 , 0.128, 0.312])

In [4]:
pxy = y
for i in range(5):
    pxy[:, i] = y[:, i]*px
    
for j in range(5):
    pxy[j, :] = pxy[j, :]/py 

In [5]:
#row represent x and column represent y: p(x|y)
pd.DataFrame(pxy)

,0,1,2,3,4
0,0.641026,0.0000,0.000000,0.000000e+00,0.000000e+00
1,0.230769,0.5625,0.333333,1.250000e-01,2.351758e-10
2,0.128205,0.3125,0.333333,3.125000e-01,1.282051e-01
3,0.000000,0.1250,0.333333,5.625000e-01,2.307692e-01
4,0.000000,0.0000,0.000000,1.172239e-12,6.410256e-01


In [6]:
x = [1,2,3,4,5]

In [7]:
y_c = [(y_1, y_2) for y_1 in range(5) for y_2 in range(5)]
y_c

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [8]:
def produce_zs(a): #convex concave function

    u2_min = 0.5
    u2_max = 5.5

    def u(x,a):
        if x>u2_max:
            return 1
        elif x<u2_min: 
            return 0
        elif a == 0:
            return (1/(u2_max-u2_min))*(x-u2_min)
        else: 
            def b(x):
                return 1-math.e**(-a*x)
            return (b(x)-b(u2_min))/(b(u2_max)-b(u2_min))
        
    ux = np.array([u(x, a) for x in x])

    z = np.zeros((5,5))
    for i in range(5):
        z[:, i] = pxy[:, i]*ux

    z = z.sum(axis=0) #z for alternative 0-4
    
    z2 = np.zeros((5,5))

    for i in range(5):
        z2[:, i] = x*pxy[:, i]

    z2 = [u(x, a) for x in list(z2.sum(axis=0))]
    return (z, z2)

In [9]:
def produce_zs2(): #sigmoid function
    def u(x): return 1/(1+math.exp(-x+2.5))
    
    ux = np.array([u(x) for x in x])

    z = np.zeros((5,5))
    for i in range(5):
        z[:, i] = pxy[:, i]*ux

    z = z.sum(axis=0) #z for alternative 0-4
    
    z2 = np.zeros((5,5))

    for i in range(5):
        z2[:, i] = x*pxy[:, i]

    z2 = [u(x) for x in list(z2.sum(axis=0))]
    return (z, z2)

In [29]:
w = [0.5, 0.5]

(z_a3, z2_a3) = produce_zs(-3) #generate z and z tilde from concave utility function
(z_, z2_) = produce_zs2() #generate z and z tilde from sigmoid function 

zs = []
z2s = []
for combo in y_c:
    i1 = combo[0]
    i2 = combo[1]

    zs.append(np.dot(w, [z_[i1], z_a3[i2]]))
    z2s.append(np.dot(w, [z2_[i1], z2_a3[i2]]))
ss.rankdata(zs) == ss.rankdata(z2s)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True])

In [30]:
diff_combo = np.array(y_c)[ss.rankdata(zs) != ss.rankdata(z2s)]

In [31]:
for combo in diff_combo:
    i1 = combo[0]
    i2 = combo[1]
    print([i1+1, i2+1])
    
    print(z_[i1])
    print(z_a3[i2])
    print("z", np.dot(w, [z_[i1], z_a3[i2]]))
    print("z2", np.dot(w, [z2_[i1], z2_a3[i2]]))


[2, 5]
0.5090819755200268
0.14566641523627705
z 0.32737419537815193
z2 0.2836794403595454
[3, 1]
0.6058581584057752
7.78357592527207e-05
z 0.30296799708251393
z2 0.31123246865292264
[3, 2]
0.6058581584057752
0.001576647228817489
z 0.3037174028172963
z2 0.3113039428012126
[3, 3]
0.6058581584057752
0.003896234406700238
z 0.3048771964062377
z2 0.31150605472870174
[3, 4]
0.6058581584057752
0.006424787528653967
z 0.30614147296721456
z2 0.31225699162817555
[3, 5]
0.6058581584057752
0.14566641523627705
z 0.3757622868210261
z2 0.33709914864106416
[4, 1]
0.7015967665957551
7.78357592527207e-05
z 0.3508373011775039
z2 0.3592999991296387
[4, 2]
0.7015967665957551
0.001576647228817489
z 0.3515867069122863
z2 0.3593714732779287
[4, 3]
0.7015967665957551
0.003896234406700238
z 0.3527465005012277
z2 0.3595735852054178
[4, 4]
0.7015967665957551
0.006424787528653967
z 0.3540107770622045
z2 0.36032452210489163


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])